In [4]:
%pylab widget
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [5]:
import fastcat.fastcat as fc
from glob import glob

## First I'll look at the detector MTF

### I'll load all of the kernels 

In [61]:
sorted(MV_detectors)

['../../fastcat/data/Detectors/CWO-261-micrometer',
 '../../fastcat/data/Detectors/CWO-392-micrometer',
 '../../fastcat/data/Detectors/CWO-784-micrometer',
 '../../fastcat/data/Detectors/CsI-010-micrometer',
 '../../fastcat/data/Detectors/CsI-150-micrometer',
 '../../fastcat/data/Detectors/CsI-336-micrometer',
 '../../fastcat/data/Detectors/CsI-392-micrometer',
 '../../fastcat/data/Detectors/CsI-784-micrometer',
 '../../fastcat/data/Detectors/CuGOS-336-micrometer',
 '../../fastcat/data/Detectors/CuGOS-392-micrometer',
 '../../fastcat/data/Detectors/CuGOS-784-micrometer']

In [6]:
spectra = glob('../../fastcat/data/MV_spectra/*.txt')
phantoms = ['XCAT','XCAT2','Catphan_515','Catphan_MTF']
MV_detectors = glob('../../fastcat/data/Detectors/*')

dispatcher={'Catphan_515':fc.Catphan_515,
            'XCAT':fc.XCAT2,
            'XCAT2':fc.XCAT,
            'Catphan_MTF':fc.Catphan_MTF}

In [ ]:
s.

In [91]:
s = fc.Spectrum()

contrasts = []
mtfs = []
label = []

nviews = 487

for phan in phantoms[:1]:
    phantom = dispatcher[phan]()
    for jj in [-1]:
        for kk in [2,-1]:
            spectrum = 'Varian_truebeam' #sorted(spectra)[jj]
            det = sorted(MV_detectors)[kk]
            print(spectrum, det)
            if jj == 1:
                s = fc.calculate_spectrum(80, 14, 3, 50)
                s.attenuate(2.2,fc.get_mu(z=13))
#             if jj == 2:
#                 s = fc.calculate_spectrum(100, 14, 3, 50)
#                 s.attenuate(2.2,fc.get_mu(z=13))
#             if jj == 3:
#                 s = fc.calculate_spectrum(120, 14, 3, 50)
#                 s.attenuate(2.2,fc.get_mu(z=2))
            else:
                s.load(spectrum.split('/')[-1].split('.')[0])
                
            kernel = fc.Kernel(s, det.split('/')[-1].split('.')[0])
            angles = np.linspace(0,np.pi*2,nviews)#480)        

            phantom.return_projs(kernel,s,angles,det_on=True,scat_on=True,mgy=18/nviews)
            phantom.reconstruct('FDK')
#             phantom.reconstruct('CGLS')
            contrasts.append(phantom.img.T)
            label.append([phan,spectrum,det])
            
#             break
#         break
#     break
    

Varian_truebeam ../../fastcat/data/Detectors/CWO-784-micrometer
(17, 25, 25) 17
bowtie is off False
This is a small edep file starting at 30
really doing it!
ratio is 96.70311643102683 number of photons 0.5881372951356504
Varian_truebeam ../../fastcat/data/Detectors/CuGOS-784-micrometer
(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 91.8386393491385 number of photons 0.02636808212587352


In [105]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 7

small_titles = ['6 MV CWO','6 MV GOS']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts):
    
    plt.subplot(1,2,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    if ii == 10:
        im = np.fliplr(np.load('../../XCAT_phantom/head_gt.npy'))
        plt.imshow(im,cmap='gray')
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        
        ax = plt.gca()
        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        im2 = axins.imshow(im,cmap='gray')
        x1, x2, y1, y2 = 60, 140, 110, 80
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')
        plt.axis('off')
        
        # get the colors of the values, according to the 
        # colormap used by imshow
        colors = [ im2.cmap(im2.norm(value)) for value in values]
        # create a patch (proxy artist) for every color 
        patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labs[i]) ) for i in range(len(values)) ]
        # put those patched as legend-handles into the legend
        plt.legend(handles=patches, bbox_to_anchor=(0.65, 1), loc=2, borderaxespad=0. )
    else:
    
        mm = np.mean(sc[375:390,395:420,index])
        mm_air = np.mean(sc[-200:,-200:,index])

        im_HU = (sc[:,:,index] - mm)/(mm_air - mm)*-1000

        im = np.rot90(im_HU,3)[230:-250,250:-250]#[270:-310,320:-320]

#         im[dist > 245] = -1000

    #     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
        plt.imshow(im,cmap='gray',vmax=500,vmin=-500)
        plt.text(-70,-20,alph[ii],color='w',fontsize=14)
        plt.axis('off')

plt.tight_layout()
plt.savefig('/home/xcite/fastcat_paper/figures/XCATs.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
plt.figure()
plt.imshow(sc[:,:,index])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …